In [1]:
import xarray as xr
import pandas as pd
import plotly.express as px

## Import data


In [2]:
station = "Papa"

In [3]:
obs_data = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr")
obs_data

<xarray.Dataset> Size: 39kB
Dimensions:    (time: 156, latitude: 3, longitude: 5, layer: 1)
Coordinates:
  * latitude   (latitude) float64 24B 48.5 49.5 50.5
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 40B -138.5 -134.5 -130.5 -129.5 -128.5
  * time       (time) datetime64[ns] 1kB 2002-01-31 2002-02-28 ... 2014-12-31
Data variables:
    day        (time, latitude, longitude, layer) float64 19kB ...
    night      (time, latitude, longitude, layer) float64 19kB ...
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [4]:
cafe_data = xr.open_dataset(f"../data/1_products/{station}_cafe.zarr", engine="zarr")
cafe_data = cafe_data.resample(time="1D").mean(dim="time").interpolate_na(dim="time", method="linear").CAFE
first_year = str(obs_data.indexes["time"].to_pydatetime()[0].year)
last_year = str(obs_data.indexes["time"].to_pydatetime()[-1].year)
cafe_data = cafe_data.sel(time=slice(first_year, last_year)).sel(
    latitude=obs_data.latitude, longitude=obs_data.longitude
)
cafe_data

<xarray.DataArray 'CAFE' (time: 4748, latitude: 3, longitude: 5)> Size: 570kB
array([[[201.98823796, 177.57917843, 156.17271104, 198.56244503,
         210.54086602],
        [169.35932402, 172.15292809, 206.4321158 , 125.94870713,
         291.12238133],
        [189.5189887 , 276.35153674, 210.15400956, 200.85998194,
         240.87817051]],

       [[203.76939011, 178.72909028, 157.42665588, 200.47350999,
         213.19496759],
        [170.31443145, 173.60992282, 209.05164146, 127.21294817,
         292.64775487],
        [190.14819471, 278.14904039, 209.91013421, 201.61239723,
         241.03391302]],

       [[205.55054225, 179.87900212, 158.68060071, 202.38457495,
         215.84906915],
        [171.26953888, 175.06691754, 211.67116712, 128.47718921,
         294.1731284 ],
        [190.77740072, 279.94654404, 209.66625887, 202.36481251,
         241.18965553]],
...
       [[170.52927712, 166.64136064, 128.24976112, 140.75744646,
         193.91424578],
        [175.70730597, 123.22247849, 128.72559501, 162.1158184 ,
         208.32162393],
        [121.17932796, 178.29756233, 118.5788749 , 189.01412831,
         220.04609514]],

       [[170.49672741, 166.26292886, 128.60009342, 138.53333749,
         192.61283196],
        [177.59693252, 123.4286626 , 127.84754528, 160.85597485,
         206.76483505],
        [122.00846036, 181.02533747, 116.89553316, 188.42934064,
         219.57084213]],

       [[170.4641777 , 165.88449707, 128.95042572, 136.30922852,
         191.31141815],
        [179.48655907, 123.6348467 , 126.96949555, 159.59613131,
         205.20804618],
        [122.83759276, 183.75311261, 115.21219141, 187.84455297,
         219.09558913]]])
Coordinates:
  * latitude   (latitude) float64 24B 48.5 49.5 50.5
  * longitude  (longitude) float64 40B -138.5 -134.5 -130.5 -129.5 -128.5
  * time       (time) datetime64[ns] 38kB 2002-01-01 2002-01-02 ... 2014-12-31
Attributes: (12/13)
    adg wavelength:               443nm
    aph wavelength:               443nm
    bbp wavelength:               443nm
    doi:                          10.1002/2016GB005521
    long_name:                    primary_production_using_CAFE_model
    mld_criteria:                 0.125 kg m^-3
    ...                           ...
    reference:                    Silsbe_et_al_2016
    remote_sensing_data_product:  OCCCIv6
    resolution:                   0.00f; // float
    standard_name:                NPP
    units:                        mg m-2 day-1
    weight:                       carbon

## Create climatology


In [5]:
cafe_data_climato = cafe_data.groupby("time.dayofyear").mean()

all_years = []
for year in range(int(cafe_data.time.min().dt.year), int(cafe_data.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = cafe_data_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
cafe_data_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
cafe_data_climato_expanded.time.attrs = cafe_data.time.attrs
cafe_data_climato_expanded.attrs["description"] = "CAFE climatology (2002-2015) interpolated to daily frequency"
cafe_data_climato_expanded

<xarray.DataArray 'CAFE' (time: 4748, latitude: 3, longitude: 5)> Size: 570kB
array([[[177.71178235, 161.41269353, 193.68857443, 196.95374495,
         196.37660467],
        [182.36528517, 179.65864853, 185.92030276, 177.32872974,
         178.98195539],
        [170.45912736, 179.5755909 , 191.56329481, 198.50811063,
         234.8607409 ]],

       [[178.10466058, 161.92034025, 193.40315871, 196.60976264,
         196.09143149],
        [182.52317943, 179.79587273, 185.23819913, 177.14548609,
         179.0974507 ],
        [170.08140623, 179.68438172, 191.6775274 , 198.67553297,
         235.30646123]],

       [[178.49753882, 162.42798697, 193.11774299, 196.26578033,
         195.80625831],
        [182.68107369, 179.93309694, 184.55609549, 176.96224245,
         179.21294601],
        [169.70368511, 179.79317255, 191.79175998, 198.84295531,
         235.75218157]],
...
       [[174.5852352 , 159.36515103, 192.6996474 , 194.17742751,
         196.95703042],
        [182.78782957, 175.89348879, 182.69534369, 181.19905981,
         172.7848946 ],
        [166.54221672, 172.05007917, 184.20807471, 197.20618246,
         231.96760643]],

       [[174.83859791, 159.75523285, 192.34472303, 193.51535489,
         196.36758682],
        [183.01760991, 175.93449679, 181.74419579, 180.82165582,
         172.6633005 ],
        [166.17987455, 172.23042934, 184.2115791 , 197.27074303,
         232.36478865]],

       [[175.09196062, 160.14531467, 191.98979865, 192.85328228,
         195.77814323],
        [183.24739026, 175.97550479, 180.7930479 , 180.44425182,
         172.54170639],
        [165.81753238, 172.41077951, 184.2150835 , 197.3353036 ,
         232.76197087]]])
Coordinates:
  * latitude   (latitude) float64 24B 48.5 49.5 50.5
  * longitude  (longitude) float64 40B -138.5 -134.5 -130.5 -129.5 -128.5
  * time       (time) datetime64[ns] 38kB 2002-01-01 2002-01-02 ... 2014-12-31
Attributes: (12/14)
    adg wavelength:               443nm
    aph wavelength:               443nm
    bbp wavelength:               443nm
    doi:                          10.1002/2016GB005521
    long_name:                    primary_production_using_CAFE_model
    mld_criteria:                 0.125 kg m^-3
    ...                           ...
    remote_sensing_data_product:  OCCCIv6
    resolution:                   0.00f; // float
    standard_name:                NPP
    units:                        mg m-2 day-1
    weight:                       carbon
    description:                  CAFE climatology (2002-2015) interpolated t...

## Plot


In [6]:
table_cafe = (
    pd.DataFrame(
        {
            "CAFE": cafe_data.mean(["latitude", "longitude"]).to_dataframe()["CAFE"],
            "CAFE_climato": cafe_data_climato_expanded.mean(["latitude", "longitude"]).to_dataframe()["CAFE"],
        }
    )
    .stack()
    .reset_index()
    .rename(columns={"level_1": "type", 0: "NPP"})
)

In [9]:
px.line(table_cafe, x="time", y="NPP", color="type", title=f"CAFE vs Climato : {station}")

## Export


In [10]:
cafe_data_climato_expanded.to_zarr(f"../data/1_products/{station}_cafe_climato.zarr", mode="w")